In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['augment2', 'jigsaw-unintended-bias-in-toxicity-classification']


In [2]:
train_filename = "../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv"
df = pd.read_csv(train_filename)

# DATA CLEANING

In [3]:
df = df.drop([
    'id', 
    'article_id',
    'rating', 
    'funny',
    'wow',
    'sad',
    'likes',
    'disagree', 
    'created_date', 
    'publication_id',
    'parent_id',
    'sexual_explicit', 
    'identity_annotator_count'
], axis = 1)

In [4]:
df = df.drop(['severe_toxicity',
    'obscene',
    'identity_attack',
    'insult', 
    'threat',
    'toxicity_annotator_count'
],axis = 1)
df.target = np.where(df['target'] >= 0.5, 1, 0)

In [5]:
df

,target,comment_text,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white
0,0,"This is so cool. It's like, 'would you want yo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Thank you!! This would make my life a lot less...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,This is such an urgent design problem; kudos t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Is this something I'll be able to install on m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,haha you guys are a bunch of losers.,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5,1,ur a sh*tty comment.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,hahahahahahahahhha suck it.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,FFFFUUUUUUUUUUUUUUU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,The ranchers seem motivated by mostly by greed...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,It was a great show. Not a combo I'd of expect...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
train = df.sample(frac=0.9, random_state=200)
supp_df = pd.read_csv("../input/augment2/aug_2.csv")
for x in ['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat'] : supp_df[x] = 0
train = train.append(supp_df)
val = df.drop(train.index)

identity_columns = ['asian', 'atheist',
       'bisexual', 'black', 'buddhist', 'christian', 'female',
       'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
       'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
       'muslim', 'other_disability', 'other_gender',
       'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']

y_identities_train = (train[identity_columns] >= 0.5).astype(int).values
y_identities_val = (val[identity_columns] >= 0.5).astype(int).values

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
from keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(df.comment_text)

Using TensorFlow backend.


In [8]:
from keras.preprocessing.text import text_to_word_sequence
def to_sequence(text):
    seq = []
    for i in text_to_word_sequence(text):
        if i in t.word_index:
            seq += [t.word_index[i]]
    return seq
tokenized_train = train.comment_text.apply(to_sequence)
tokenized_val = val.comment_text.apply(to_sequence)

In [9]:
tokenized_train

1732350    [35, 158, 6, 161, 5, 81, 48, 42, 63, 826, 8, 1...
1063992       [432, 5, 5613, 447, 217, 15, 164, 2281, 2, 39]
481539     [1235, 1325, 9, 8795, 133, 950, 22375, 3662, 2...
838844              [80, 12, 335, 15, 28283, 9, 153, 40, 12]
859495     [580, 2771, 4448, 381, 540, 7, 2790, 540, 162,...
708598                                   [81, 2965, 2243, 8]
1081091    [35, 13, 2321, 52, 1, 1660, 19, 1, 490, 128, 6...
261509     [11, 1779, 35, 260, 6, 3206, 1598, 5, 1497, 77...
256010     [337, 266, 48472, 253, 6617, 151989, 31350, 34...
1726484    [58, 26, 12, 90117, 1, 388566, 40, 128, 379, 1...
344465     [1, 2048, 556, 2016, 24782, 7, 1, 176, 63, 17,...
1535402    [9, 5, 1909, 37, 43, 552, 25850, 13387, 9, 89,...
323949     [1, 84, 448, 43, 1, 1561, 4, 271, 7, 1, 2905, ...
683638     [18, 6, 136, 2, 39, 19, 6334, 23, 83, 632, 2, ...
106522     [75, 2862, 15, 558, 1574, 275, 221, 64, 12184,...
173175        [33, 30, 6, 1, 4870, 3054, 192, 27, 8, 9, 136]
1062616    [722, 328, 12

In [10]:
print('Maximum text length: {}'.format(
len(max((df.comment_text), key=len))))
print('Minimum text length: {}'.format(
len(min((df.comment_text), key=len))))

Maximum text length: 1906
Minimum text length: 1


In [11]:
max_words = 1000
vocab_size = len(t.word_index) + 1

In [12]:
from keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(tokenized_train, maxlen=max_words)
X_val = pad_sequences(tokenized_val, maxlen=max_words)

In [13]:
Y_train = train.target
Y_val = val.target

# CNN-LSTM

In [14]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

In [15]:
from keras.regularizers import l2
embedding_size=32
model5 = tf.keras.Sequential()
model5.add(layers.Embedding(vocab_size, embedding_size, input_length=max_words))
model5.add(layers.Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
model5.add(layers.MaxPooling1D(pool_size=5))
model5.add(layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model5.add(layers.MaxPooling1D(pool_size=5))
model5.add(layers.Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model5.add(layers.MaxPooling1D(pool_size=2))
model5.add(layers.LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model5.add(layers.Dense(1, activation='sigmoid'))
print(model5.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 32)          12726688  
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 128)         8320      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 200, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 200, 128)          49280     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 40, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)    

In [16]:
model5.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [17]:
model5.fit(X_train, Y_train, epochs=3, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
 690432/1628966 [===========>..................] - ETA: 8:14 - loss: 0.1492 - acc: 0.9451

In [18]:
train_acc = model5.evaluate(X_train, Y_train)
train_pred = model5.predict(X_train)
val_acc = model5.evaluate(X_val, Y_val)
val_pred = model5.predict(X_val)

1193472/1628966 [====================>.........] - ETA: 4:45 - loss: 0.0906 - acc: 0.9642

In [19]:
print(train_acc)
print(val_acc)

[0.090653152544501, 0.96420366]
[0.1348135188947888, 0.94606936]


# TEST EVAL

In [20]:
test_filename = "../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv"
df_test = pd.read_csv(test_filename)

In [21]:
tokenized_test = df_test.comment_text.apply(to_sequence)
X_test = pad_sequences(tokenized_test, maxlen=max_words)

In [22]:
Y_test = model5.predict(X_test)

In [23]:
submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv', index_col='id')
submission['prediction'] = Y_test
submission.to_csv('submission.csv')

### AUC SCORE

In [24]:
from sklearn.metrics import roc_auc_score

def power_mean(x, p=2):
    return np.power(np.mean(np.power(x, p)),1/p)

def get_s_auc(y_true,y_pred,y_identity):
    mask = y_identity==1
    try:
        s_auc = roc_auc_score(y_true[mask],y_pred[mask])
    except:
        s_auc = 1
    return s_auc

def get_bpsn_auc(y_true,y_pred,y_identity):
    mask = (y_identity==1) & (y_true==0) | (y_identity==0) & (y_true==1)
    try:
        bpsn_auc = roc_auc_score(y_true[mask],y_pred[mask])
    except:
        bpsn_auc = 1
    return bpsn_auc

def get_bspn_auc(y_true,y_pred,y_identity):
    mask = (y_identity==1) & (y_true==1) | (y_identity==0) & (y_true==0)
    try:
        bspn_auc = roc_auc_score(y_true[mask],y_pred[mask])
    except:
        bspn_auc = 1
    return bspn_auc

def get_total_auc(y_true,y_pred,y_identities):

    N = y_identities.shape[1]
    
    saucs = np.array([get_s_auc(y_true,y_pred,y_identities[:,i]) for i in range(N)])
    bpsns = np.array([get_bpsn_auc(y_true,y_pred,y_identities[:,i]) for i in range(N)])
    bspns = np.array([get_bspn_auc(y_true,y_pred,y_identities[:,i]) for i in range(N)])

    M_s_auc = power_mean(saucs)
    M_bpsns_auc = power_mean(bpsns)
    M_bspns_auc = power_mean(bspns)
    rauc = roc_auc_score(y_true,y_pred)


    total_auc = M_s_auc + M_bpsns_auc + M_bspns_auc + rauc
    total_auc/= 4

    return total_auc

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 12] Cannot allocate memory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [25]:
print(get_total_auc(Y_train, train_pred, y_identities_train))
print(get_total_auc(Y_val, val_pred, y_identities_val))

0.9588675455320481
0.9220339870498598


# EXPORT FOR ERROR ANALYSIS

In [26]:
Y_val_check = model5.predict(X_val)

In [27]:
val['pred'] = Y_val_check

In [28]:
false_pos = val[(val["target"] == 0) & (val["pred"] > 0.5)]

In [29]:
false_neg = val[(val["target"] == 1) & (val["pred"] < 0.5)]

In [30]:
false_pos.to_csv('false_pos.csv')
false_neg.to_csv('false_neg.csv')